<a href="https://colab.research.google.com/github/Data-Science-and-Data-Analytics-Courses/UCSanDiegoX---Machine-Learning-Fundamentals-03-Jan-2019-audit/blob/master/Packages/Git.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os, shutil
from google.colab import drive, files
from urllib.parse import urlsplit
from pathlib import Path
import json
from getpass import getpass

# SSH

In [0]:
def ssh(keyfile=""):
  """
  Add private key to ssh-agent
  keyfile: path to private key file
  """
  
  sshdir = sshdir or input("Enter path to .ssh directory: ")
  
  shutil.rmtree("/root/.ssh")
  shutil.copytree(sshdir, "/root/.ssh")
  !chmod 700 /root/.ssh
  
  !ssh-keyscan github.com >> /root/.ssh/known_hosts
  !chmod 644 /root/.ssh/known_hosts
  !eval "$(ssh-agent -s)"
  !ssh-add ~/.ssh/id_rsa
  


file:/root/.gitconfig	user.name=vinh


In [0]:
!ssh-agent -s


SSH_AUTH_SOCK=/tmp/ssh-LMA7smJYffOU/agent.463; export SSH_AUTH_SOCK;
SSH_AGENT_PID=464; export SSH_AGENT_PID;
echo Agent pid 464;


# Configure

In [0]:
def config(confile="", level="--global"):
  """
  Configure Git
  confile: path to .json configuration file, containing text: {"email": "<email>", "name": "<display name>", ...}
  level: -—local|--global|--system
  """

  # Configurations
  confile = confile or input("Enter path to .json configuration file: ")
  with open(confile) as f:
    configs = json.load(f)

  # Configure
  email = configs["email"]
  name = configs["name"]
  !git config "$level" user.email "$email"
  !git config "$level" user.name "$name"

# Clone

In [0]:
def clone(url, dest=".", branch="master", reloc=True):
  """
  Clone remote branch from url into dest
  branch not provided: clone all branches
  reloc is True: relocate to repository
  """

  url = url.strip(" /")
  repo = Path(dest, os.path.basename(url)).resolve()

  # dest must not be inside existing repository
  is_out = !git -C "$dest" rev-parse
  if not is_out: # inside repository
    raise ValueError("Can't clone into existing repository")
  
  # Clone
  p = repo.as_posix()
  if branch: # specific branch
    !git clone --single-branch "$url" -b "$branch" "$p"
  else: # all branches
    !git clone "$url" "$p"
  
  # Relocate
  if reloc:
    %cd "$repo"

  return repo.as_posix()
  
if __name__ == "__main__":
  url = "https://github.com/Data-Science-and-Data-Analytics-Courses/UCSanDiegoX---Machine-Learning-Fundamentals-03-Jan-2019-audit"
  clone(url)

Cloning into '/content/UCSanDiegoX---Machine-Learning-Fundamentals-03-Jan-2019-audit'...
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 887 (delta 2), reused 0 (delta 0), pack-reused 879
Receiving objects: 100% (887/887), 3.38 MiB | 10.11 MiB/s, done.
Resolving deltas: 100% (451/451), done.
/content/UCSanDiegoX---Machine-Learning-Fundamentals-03-Jan-2019-audit


# Push

In [0]:
def push(url, branch="master", logfile=""):
  """
  Push branch to url
  branch: not provided, push all branches
  logfile: path to .json log-in file, containing text: {"username": "<username>", "password": "<password>"}
  """
  
  rurl = urlsplit(url)

  # Log-in information
  logfile = logfile or input("Enter path to .json log-in file: ")
  with open(logfile) as f:
    login = json.load(f)

  # Construct authenticated remote
  rauth = rurl._replace(netloc="{username}:{password}@{hostname}".format(username=login["username"], password=login["password"], hostname=rurl.hostname)) # add username and password
  
  # Push
  remote = rauth.geturl()
  if branch:
    !git push "$remote" "$branch"
  else:
    !git push --all "$remote"

if __name__ == "__main__":
  url = "https://github.com/Data-Science-and-Data-Analytics-Courses/UCSanDiegoX---Machine-Learning-Fundamentals-03-Jan-2019-audit"
  push(url)